# PLN I

## Desafio 1

Se utilizará la notebook planteada en clase de base para realizar el desafío.

Importamos las bilbiotecas que se utilizarán en todo el desafío

In [10]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

### Punto 1: similaridad entre documentos

Cargo y dividos los datos en train y test

In [11]:
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

Instanciamos un vectorizador. Utilizo los parámetros:
- "stop_words='english'": para no considerar palabras comunes y poco informativas, 
- "max_df=0.85": para eliminar términos que aparezcan en más del 85% de los documentos

In [12]:
tfidfvect = TfidfVectorizer(stop_words='english',max_df=0.85)

Luego, genero el set de entrenamiento a partir del vectorizador

In [13]:
X_train = tfidfvect.fit_transform(newsgroups_train.data)
y_train = newsgroups_train.target

A continuación, obtengo 5 documentos arbitrareamente y, para cada uno de ellos:
- Mido su similaridad coseno contra todo los elementos del set
- Me quedo con los 5 más cercanos
- Los muestro para poder evaluarlos

In [29]:
documents_to_analyze_idx = [33,551,3123,11045,5312]

Defino function para poder calcular la similaridad coseno y obtener los 5 documentos más cercanos. Para cada uno de ellos, muestro la comparo la clase a la cual pertenecen e imprimo su contenido para evaluar.

In [54]:
def show_nearest_five(baseDocumentIdx):
    cossim = cosine_similarity(X_train[baseDocumentIdx], X_train)[0]
    mostsim = np.argsort(cossim)[::-1][1:6] # no considero al propio documento, por eso arranco de 1
    
    print("---------------------------\n")
    print("Clase de documento base      :",newsgroups_train.target_names[y_train[baseDocumentIdx]])
    for i,similarDocumentIdx in enumerate(mostsim):
        print(f"Clase de documento cercano {i} :",newsgroups_train.target_names[y_train[similarDocumentIdx]])

    print("----------------------------------")
    print("\nDocBase: \n",newsgroups_train.data[baseDocumentIdx])

    for i,similarDocumentIdx in enumerate(mostsim):
        print("----------------------------------")
        print(f"\nDoc cercano {i} :\n",newsgroups_train.data[similarDocumentIdx])

    

Luego, vemos los resultados para los 5 documentos definidos

In [55]:
for i in documents_to_analyze_idx:
 show_nearest_five(i)

---------------------------

Clase de documento base      : talk.politics.mideast
Clase de documento cercano 0 : talk.politics.mideast
Clase de documento cercano 1 : talk.politics.mideast
Clase de documento cercano 2 : talk.politics.mideast
Clase de documento cercano 3 : sci.electronics
Clase de documento cercano 4 : talk.politics.mideast
----------------------------------

DocBase: 
 
How do you define war?  Do seiges and constant attacks on villiages
count as acts of war, or is that only when the Jews do them?
January, 1948: Arab Liberation Army attacks Kfar Szold
               1000 men attack Kfar Etzion, 14 miles south of Jerusalem,
                    after cutting off the supply lines to it.
Attacks on Yehiam (Western Galilee) and kibbutz Tirat Tzvi.
By Mid-March, The Jewish settlements in the Negev had been cut off from
      land links with the rest of the Jewish population.
         The Etzion group of villiages, near Hebron, had been cut off,
            while 42 members of 

Comentarios:

A primera vista se observa un buen resultado de calcular la similaridad coseno entre documentos. Se observa un resultado imperfecto, pero que verifica un porcentaje de aciertos no menor para el número de clases que existen. Adicionalmente, para muchos de los casos en los que no hay aciertos, las clases de los documentos detectados como "similares" tienen contenidos que comparten cierta cercanía con el documento base, a pesar de no ser de la misma clase exacta.